# LAB 21_ADVANCED SQL

## Challenge 1 - Most Profiting Authors

In this challenge let's have a close look at the bonus challenge of the previous SQL SELECT lab -- who are the top 3 most profiting authors? Even if you have solved or think you have solved that problem in the previous lab, please still complete this challenge because the step-by-step guidances are helpful to train your problem-solving thinking.

In [4]:
# Import pandas and define the location and access (sqlite) of the database

import pandas as pd
db_url = 'sqlite:///publications.db'

## Step 1: Calculate the royalties of each sales for each author

Write a SELECT query to obtain the following output:

- Title ID
- Author ID
- Royalty of each sale for each author

The formula is:

sales_royalty = titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100

Note that titles.royalty and titleauthor.royaltyper are divided by 100 respectively because they are percentage numbers instead of floats.

In [5]:
# Select interested items
# Calculate the royalties per title by suming the royalties (prices * qty * royaltyper * royalty)
query = '''
SELECT a.au_id as AUTHOR_ID, a.au_lname as LAST_NAME, a.au_fname as FIRST_NAME, t.title as TITLE, t.price * s.qty * ta.royaltyper / 100 * t.royalty / 100 as TOTAL_ROY
FROM sales s INNER JOIN titles t on t.title_id = s.title_id INNER JOIN titleauthor ta on t.title_id = ta.title_id
INNER JOIN authors a on a.au_id = ta.au_id
ORDER BY AUTHOR_ID
'''

In [6]:
# Call the query from the data base with pandas

df = pd.read_sql(query, db_url)
df

,AUTHOR_ID,LAST_NAME,FIRST_NAME,TITLE,TOTAL_ROY
0,172-32-1176,White,Johnson,Prolonged Data Deprivation: Four Case Studies,29.985
1,213-46-8915,Green,Marjorie,The Busy Executive's Database Guide,3.998
2,213-46-8915,Green,Marjorie,The Busy Executive's Database Guide,7.996
3,213-46-8915,Green,Marjorie,You Can Combat Computer Stress!,25.116
4,238-95-7766,Carson,Cheryl,But Is It User Friendly?,110.160
5,267-41-2394,O'Leary,Michael,Cooking with Computers: Surreptitious Balance ...,11.950
6,267-41-2394,O'Leary,Michael,"Sushi, Anyone?",8.994
7,274-80-9391,Straight,Dean,Straight Talk About Computers,29.985
8,409-56-7008,Bennet,Abraham,The Busy Executive's Database Guide,5.997
9,409-56-7008,Bennet,Abraham,The Busy Executive's Database Guide,11.994


## Step 2: Aggregate the total royalties for each title for each author

Using the output from Step 1, write a query to obtain the following output:

- Title ID
- Author ID
- Aggregated royalties of each title for each author

In [9]:
# From previous query, add a Group by author and title and Sum the royalty to get the royalties per book
query2 = '''
SELECT a.au_id as AUTHOR_ID, a.au_lname as LAST_NAME, a.au_fname as FIRST_NAME, t.title as TITLE, sum(t.price * s.qty * ta.royaltyper / 100 * t.royalty / 100) as TOTAL_ROY
FROM sales s INNER JOIN titles t on t.title_id = s.title_id INNER JOIN titleauthor ta on t.title_id = ta.title_id
INNER JOIN authors a on a.au_id = ta.au_id
GROUP BY AUTHOR_ID, LAST_NAME,FIRST_NAME, TITLE
ORDER BY AUTHOR_ID
'''

In [10]:
# Call the query from the data base with pandas

df2 = pd.read_sql(query2, db_url)
df2

,AUTHOR_ID,LAST_NAME,FIRST_NAME,TITLE,TOTAL_ROY
0,172-32-1176,White,Johnson,Prolonged Data Deprivation: Four Case Studies,29.985
1,213-46-8915,Green,Marjorie,The Busy Executive's Database Guide,11.994
2,213-46-8915,Green,Marjorie,You Can Combat Computer Stress!,25.116
3,238-95-7766,Carson,Cheryl,But Is It User Friendly?,110.160
4,267-41-2394,O'Leary,Michael,Cooking with Computers: Surreptitious Balance ...,11.950
5,267-41-2394,O'Leary,Michael,"Sushi, Anyone?",8.994
6,274-80-9391,Straight,Dean,Straight Talk About Computers,29.985
7,409-56-7008,Bennet,Abraham,The Busy Executive's Database Guide,17.991
8,427-17-2319,Dull,Ann,Secrets of Silicon Valley,50.000
9,472-27-2349,Gringlesby,Burt,"Sushi, Anyone?",8.994


## Step 3: Calculate the total profits of each author

Now that each title has exactly one row for each author where the advance and royalties are available, we are ready to obtain the eventual output. Using the output from Step 2, write a query to obtain the following output:

- Author ID
- Profits of each author by aggregating the advance and total royalties of each title

Sort the output based on a total profits from high to low, and limit the number of rows to 3.

In [11]:
# From previous query, add the advance quantity per book
# Create a sub query to Group by author (so the authors that have more than one group are grouped) and sum royalties and advanced earnings
query3 = '''
with profit_authors as
(SELECT a.au_id as AUTHOR_ID, a.au_lname as LAST_NAME, a.au_fname as FIRST_NAME, t.title as TITLE, t.advance * ta.royaltyper / 100
as TOTAL_ADV, sum(t.price * s.qty * ta.royaltyper / 100 * t.royalty / 100) as TOTAL_ROY
FROM sales s INNER JOIN titles t on t.title_id = s.title_id INNER JOIN titleauthor ta on t.title_id = ta.title_id
INNER JOIN authors a on a.au_id = ta.au_id
GROUP BY AUTHOR_ID, LAST_NAME,FIRST_NAME, TITLE, TOTAL_ADV
ORDER BY t.title)
SELECT AUTHOR_ID, LAST_NAME, FIRST_NAME, ROUND(SUM(TOTAL_ADV + TOTAL_ROY),1) as BENEFIT
FROM profit_authors
GROUP BY AUTHOR_ID
ORDER BY BENEFIT DESC
LIMIT 3
'''

In [12]:
# Call the query from the data base with pandas

df3 = pd.read_sql(query3, db_url)
df3

,AUTHOR_ID,LAST_NAME,FIRST_NAME,BENEFIT
0,213-46-8915,Green,Marjorie,12162.1
1,722-51-5454,DeFrance,Michel,11271.5
2,998-72-3567,Ringer,Albert,7225.0


## Challenge 2 - Create a new table

Elevating from your solution in Challenge 1 , create a table named most_profiting_authors to hold the data about the most profiting authors. The table should have 2 columns:

- au_id - Author ID
- profits - The profits of the author aggregating the advances and royalties

In [19]:
# Create table most_profiting_authors
query4 ='''
CREATE TABLE "most_profiting_authors"(
"au_id"	varchar(11) NOT NULL, 
"profits" decimal(19, 4) DEFAULT NULL,
CONSTRAINT "titleauthor_ibfk_7" FOREIGN KEY("au_id") REFERENCES "authors"("au_id") ON DELETE CASCADE ON UPDATE CASCADE
)
'''
pd.read_sql(query4, db_url)

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [21]:
# Insert values from the previous Challenge final query
query5 ='''
INSERT INTO most_profiting_authors
with profit_authors as
(SELECT a.au_id as AUTHOR_ID, a.au_lname as LAST_NAME, a.au_fname as FIRST_NAME, t.title as TITLE, t.advance * ta.royaltyper / 100
as TOTAL_ADV, sum(t.price * s.qty * ta.royaltyper / 100 * t.royalty / 100) as TOTAL_ROY
FROM sales s INNER JOIN titles t on t.title_id = s.title_id INNER JOIN titleauthor ta on t.title_id = ta.title_id
INNER JOIN authors a on a.au_id = ta.au_id
GROUP BY AUTHOR_ID, LAST_NAME,FIRST_NAME, TITLE, TOTAL_ADV
ORDER BY t.title)
SELECT AUTHOR_ID, ROUND(SUM(TOTAL_ADV + TOTAL_ROY),1) as BENEFIT
FROM profit_authors
GROUP BY AUTHOR_ID
ORDER BY BENEFIT DESC
LIMIT 3
'''
pd.read_sql(query5, db_url)

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [22]:
# Check if the table has been created and its content
query6 = '''
SELECT *
FROM most_profiting_authors
'''
pd.read_sql(query6, db_url)

,au_id,profits
0,213-46-8915,12162.1
1,722-51-5454,11271.5
2,998-72-3567,7225.0
